### Part 3: The Logistics Commander

##### *Import Libraries*

In [ ]:
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.router import pick_model
from utils.logging_utils import log_llm_call
import re

##### *Load data  from incidents.txt*

In [2]:
input_path = '../data/incidents.txt'

try:
    with open(input_path, 'r') as f:
        lines = f.readlines()
        headers = lines[0].strip()
        incidents = [line.strip() for line in lines[1:] if line.strip()]
    print(f"{len(incidents)} incidents loaded")
except FileNotFoundError:
    print("File not found")
    scenarios = []

3 incidents loaded


##### *Setup LLM Client*

In [3]:
model = pick_model('openai', 'general')
client = LLMClient('openai', model)

##### *Step A: CoT Scoring*

In [ ]:
scored_results = []

scoring_rules = """
Analyze the incident and calculate a score (0-10) strictly using these rules:
    - Start with Base Score: 5
    - Check the 'Ages' column. If ANY age is > 60 OR < 5, add +2.
    - Check 'Main_Need' and 'Message'; 
            - If 'Rescue' or 'Flood' or life threat add +3.
            - If 'Insulin' or 'Medicine' add +1.
    - Max Score is 10.
    - Output Format: "Score: Number/10".
"""

for incident in incidents:
    prompt_text, spec = render(
                                'cot_reasoning.v1',
                                role='Triage Officer',
                                problem=f"Calculate priority score for this row: {incident}",
                                context=f"Headers: {headers}\n\nRules:\n{scoring_rules}",
                                format="Final Answer: Score: [Number]/10"
                            )

    response = client.chat([{'role': 'user', 'content': prompt_text}], temperature=0.0)
    output = response['text'].strip()

    match = re.search(r"(?:Score|Answer):\s*(\d+)", output)
    if match:
        final_score = match.group(1)
        final_score_str = f"Score: {final_score}/10"
    else:
        final_score_str = "Score: ?/10"
    
    scored_results.append(f"Incident: {incident} | {output} | {final_score_str}")
    
    print(f"Incident: {incident}")
    print(f"{'-'*150}")
    print(f"Result: \n{output}")
    print(f"{final_score_str}")
    log_llm_call('openai', model, 'cot_scoring', response['latency_ms'], response['usage'])

Incident: 1  | 08:00 AM| Gampaha | 4      | 20-40  | Water     | "Thirsty but safe on roof. Water level stable."
------------------------------------------------------------------------------------------------------------------------------------------------------
Result: 
1. Identify the components of the row: 
   - Time: 08:00 AM
   - Location: Gampaha
   - Severity Level: 4
   - Age Group: 20-40
   - Need: Water
   - Situation: "Thirsty but safe on roof. Water level stable."

2. Determine the priority score based on the severity level and situation:
   - Severity Level: 4 indicates a moderate level of urgency.
   - The individual is "thirsty but safe," which suggests that while they need water, they are not in immediate danger.
   - The water level being stable indicates no immediate threat to life.

3. Assign a score based on the above factors:
   - Severity Level: 4 contributes a score of 4.
   - Safety status (safe on roof) suggests a lower urgency, potentially reducing the score.

##### *Step B: ToT Strategy*

In [20]:
incidents_data = "\n".join(scored_results)

problem = f"""
                You are the Logistics Commander at Ragama (Starting Point).
                Incidents to attend (with scores): {incidents_data}

                Task: Compare these 3 branches and select the best route to maximize Total Priority Score saved in minimum time.
                Branch 1 (Greedy): Go to the incident with the Highest Score first.
                Branch 2 (Speed): Go to the Closest location first (Ragama/Ja-Ela).
                Branch 3 (Logictics): Go to Gampaha first.

                Assume the boat has unlimited capacity but must stop at each location to handle the incident.
            """

constraints = f""" 
                Travel Constraints:
                - Ragama (Start) -> Ragama Incident: 0 mins
                - Ragama -> Ja-Ela: 10 mins
                - Ja-Ela -> Gampaha: 40 mins
                Total Path: Ragama -> Ja-Ela -> Gampaha is 50 mins
            """

prompt_text, spec = render(
                            'tot_reasoning.v1',
                            role='Logistics Commander',
                            problem=problem,
                            branches="3 (Greedy, Speed, Logictics)",
                            constraints=constraints
                        )

response_tot = client.chat([{'role': 'user', 'content': prompt_text}], temperature=spec.temperature)

print("Optimal Route:\n")
print(response_tot['text'])

log_llm_call('openai', model, 'tot_strategy', response_tot['latency_ms'], response_tot['usage'])

Optimal Route:

To solve the problem of prioritizing incidents while maximizing the total priority score and minimizing time, let's explore the three distinct solution paths: Greedy, Speed, and Logistics.

### Branch 1: Greedy Approach
**Hypothesis:** The best strategy is to attend to the incident with the highest priority score first.

**Steps:**
1. **Identify the scores for each incident:**
   - Incident 1 (Gampaha): Score 4
   - Incident 2 (Ja-Ela): Score 3
   - Incident 3 (Ragama): Score 8
2. **Order the incidents by score:**
   - 1st: Incident 3 (Ragama, Score 8)
   - 2nd: Incident 1 (Gampaha, Score 4)
   - 3rd: Incident 2 (Ja-Ela, Score 3)
3. **Plan the route:**
   - Start at Ragama (08:20 AM incident).
   - Move to Gampaha (08:00 AM incident).
   - Finally, head to Ja-Ela (08:15 AM incident).

**Intermediate Check:**
- Total Priority Score: 8 + 4 + 3 = 15
- Total Time Taken: 08:20 AM (Ragama) → 08:00 AM (Gampaha) → 08:15 AM (Ja-Ela)

### Branch 2: Speed Approach
**Hypothesis:** 